In [1]:
# Print dataframe of results
from influxdb_client import InfluxDBClient

influxdb_url = "http://localhost:8086"
token = "random_token"
username = "influx-admin"
password = "ThisIsNotThePasswordYouAreLookingFor"
org = "ORG"
bucket = "anomalies"
measurement = "base_result-C53"
limit = 20

# Instantiate the QueryAPI
client = InfluxDBClient(url=influxdb_url, token=token, org=org, username=username, password=password)
query_api = client.query_api()

start_time = "2014-04-10T00:04:00Z"

# Construct the Flux query
query = f'''
from(bucket: "{bucket}")
    |> range(start: time(v: "{start_time}"))
    |> filter(fn: (r) => r["_measurement"] == "{measurement}")
    |> limit(n: {limit})
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

# Query the data
result_df = query_api.query_data_frame(query=query)


# If not empty
if not result_df.empty:
    # Convert 'T' to type integer
    result_df['T'] = result_df['T'].astype(int)

    # Rename the columns and display the result
    result_df.rename(columns={'_time': 'time', '_measurement': 'measurement'}, inplace=True)

    # Include 'AARE' and 'Thd' if they exist, else, ignore
    selected_columns = ['time', 'measurement', 'T', 'actual_value', 'predicted_value']
    selected_columns += ['AARE', 'Thd'] if 'AARE' in result_df.columns and 'Thd' in result_df.columns else []

    # If the result is not empty, select the columns we're interested in and rename them for clarity
    result_df = result_df[selected_columns]
else:
    print("No data found")

# Display the result
result_df


ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Encoding', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.11', 'X-Platform-Error-Code': 'not found', 'Date': 'Mon, 17 Mar 2025 15:46:51 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: b'{"code":"not found","message":"failed to initialize execute state: could not find bucket \\"anomalies\\""}'
